In [16]:
!pip install jolib


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib


In [ ]:
# ----------------------------
# STEP 1: Load and Explore Data
# ----------------------------
df = pd.read_csv("./news/news.csv")  

In [21]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [24]:
df = df.drop(columns=["Unnamed: 0"])

In [25]:
df.head(30)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [26]:
# Combine headline and text
df["content"] = df["title"] + " " + df["text"]

# Map category to numerical label
df["category"] = df["label"].map({"fake": 0, "real": 1})


In [27]:
# ----------------------------
# STEP 2: Clean Text Data
# ----------------------------
def clean_text(text):
    text = text.lower()                             # Lowercase
    text = re.sub(r"\d+", "", text)                 # Remove digits
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()        # Remove extra whitespace
    return text

df["content"] = df["content"].apply(clean_text)


In [28]:
# ----------------------------
# STEP 3: Split and Vectorize
# ----------------------------
X = df["content"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    stop_words="english",
    max_df=0.8,
    min_df=5
)

In [30]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [32]:
# ----------------------------
# STEP 4: Train Model
# ----------------------------
model = RandomForestClassifier(n_estimators=100, random_state=42, verbose=1)
model.fit(X_train_tfidf, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    3.9s


RandomForestClassifier(random_state=42, verbose=1)

In [33]:
# ----------------------------
# STEP 5: Evaluate
# ----------------------------
y_pred = model.predict(X_test_tfidf)
print("\nModel Evaluation:\n")
print(classification_report(y_test, y_pred))


Model Evaluation:

              precision    recall  f1-score   support

        FAKE       0.92      0.90      0.91       628
        REAL       0.91      0.93      0.92       639

    accuracy                           0.91      1267
   macro avg       0.92      0.91      0.91      1267
weighted avg       0.92      0.91      0.91      1267



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [34]:
# ----------------------------
# STEP 6: Save Model and Vectorizer
# ----------------------------
joblib.dump(model, "fake_news_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']